In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [8]:
df = pd.read_csv('../data/raw/data.csv')

In [20]:
# Make sure date column is datetime
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

# Define reference point for recency
latest_date = df['TransactionStartTime'].max()

# Group by customer_id to calculate RFM values
rfm_df = df.groupby('CustomerId').agg({
    'TransactionStartTime': lambda x: (latest_date - x.max()).days,   # Recency
    'TransactionId': 'count',                        # Frequency
    'Value': 'sum'                                   # Monetary
}).reset_index()

# Rename columns
rfm_df.columns = ['CustomerId', 'Recency', 'Frequency', 'Monetary']
rfm_df.head()

,CustomerId,Recency,Frequency,Monetary
0,CustomerId_1,83,1,10000
1,CustomerId_10,83,1,10000
2,CustomerId_1001,89,5,30400
3,CustomerId_1002,25,11,4775
4,CustomerId_1003,11,6,32000


In [21]:
scaler = StandardScaler()
scaled_rfm = scaler.fit_transform(rfm_df[['Recency', 'Frequency', 'Monetary']])
scaled_rfm

array([[ 1.93760472, -0.25345907, -0.08952358],
       [ 1.93760472, -0.25345907, -0.08952358],
       [ 2.15888193, -0.21218649, -0.08201119],
       ...,
       [-1.1233968 ,  0.77835538,  0.13293045],
       [ 1.34753213, -0.08836876, -0.03759971],
       [-1.1233968 , -0.03677804, -0.03318065]])

In [22]:
kmeans = KMeans(n_clusters=3, random_state=42)
rfm_df['Cluster'] = kmeans.fit_predict(scaled_rfm)
rfm_df

,CustomerId,Recency,Frequency,Monetary,Cluster
0,CustomerId_1,83,1,10000,0
1,CustomerId_10,83,1,10000,0
2,CustomerId_1001,89,5,30400,0
3,CustomerId_1002,25,11,4775,2
4,CustomerId_1003,11,6,32000,2
...,...,...,...,...,...
3737,CustomerId_992,4,6,32000,2
3738,CustomerId_993,25,5,32000,2
3739,CustomerId_994,0,101,614077,2
3740,CustomerId_996,67,17,151000,0


In [23]:
# Get cluster characteristics
cluster_summary = rfm_df.groupby('Cluster').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean'
}).round(2)

print("Cluster Summary:\n", cluster_summary)

# Find the cluster with lowest Frequency and Monetary (highest Recency helps too)
high_risk_cluster = cluster_summary.sort_values(
    by=['Frequency', 'Monetary', 'Recency'],
    ascending=[True, True, False]  # low F, low M, high R = worst
).index[0]

print("High-Risk Cluster:", high_risk_cluster)


Cluster Summary:
          Recency  Frequency      Monetary
Cluster                                  
0          60.72       7.69  8.925166e+04
1          28.00    4091.00  1.049000e+08
2          11.64      34.92  3.096850e+05
High-Risk Cluster: 0


In [24]:
rfm_df['is_high_risk'] = (rfm_df['Cluster'] == high_risk_cluster).astype(int)
rfm_df

,CustomerId,Recency,Frequency,Monetary,Cluster,is_high_risk
0,CustomerId_1,83,1,10000,0,1
1,CustomerId_10,83,1,10000,0,1
2,CustomerId_1001,89,5,30400,0,1
3,CustomerId_1002,25,11,4775,2,0
4,CustomerId_1003,11,6,32000,2,0
...,...,...,...,...,...,...
3737,CustomerId_992,4,6,32000,2,0
3738,CustomerId_993,25,5,32000,2,0
3739,CustomerId_994,0,101,614077,2,0
3740,CustomerId_996,67,17,151000,0,1


In [26]:
df_processed = pd.read_csv('../data/processed/data_processed.csv')
df_labeled = df_processed.merge(rfm_df[['CustomerId', 'is_high_risk']], on='CustomerId', how='left')

In [28]:
df_labeled[['CustomerId', 'is_high_risk']].head()


,CustomerId,is_high_risk
0,CustomerId_4406,0
1,CustomerId_4406,0
2,CustomerId_4683,1
3,CustomerId_988,0
4,CustomerId_988,0


In [29]:
df_labeled.to_csv("../data/processed/data_labeled.csv", index=False)